In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

from src.paths import TRANSFORMED_DATA_DIR 

df = pd.read_parquet(TRANSFORMED_DATA_DIR/ 'tabular_data1.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89300,3.0,0.0,2.0,3.0,2.0,3.0,13.0,8.0,9.0,9.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
89301,6.0,4.0,0.0,0.0,2.0,0.0,14.0,7.0,8.0,4.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
89302,7.0,2.0,3.0,4.0,7.0,4.0,10.0,9.0,7.0,11.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
89303,6.0,5.0,4.0,3.0,0.0,3.0,11.0,12.0,9.0,10.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 7, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(40545, 674)
y_train.shape=(40545,)
X_test.shape=(48760, 674)
y_test.shape=(48760,)


In [4]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-04-02 10:17:37,181] A new study created in memory with name: no-name-1ae88b58-c104-41c4-9c69-638c822f27a3


[LightGBM] [Warning] feature_fraction is set=0.5678478718344357, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5678478718344357
[LightGBM] [Warning] bagging_fraction is set=0.580858477264599, subsample=1.0 will be ignored. Current value: bagging_fraction=0.580858477264599
[LightGBM] [Warning] feature_fraction is set=0.5678478718344357, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5678478718344357
[LightGBM] [Warning] bagging_fraction is set=0.580858477264599, subsample=1.0 will be ignored. Current value: bagging_fraction=0.580858477264599


[I 2023-04-02 10:17:59,750] Trial 0 finished with value: 3.472777194988616 and parameters: {'num_leaves': 152, 'feature_fraction': 0.5678478718344357, 'bagging_fraction': 0.580858477264599, 'min_child_samples': 43}. Best is trial 0 with value: 3.472777194988616.


[LightGBM] [Warning] feature_fraction is set=0.6655715429753413, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6655715429753413
[LightGBM] [Warning] bagging_fraction is set=0.921814321108279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.921814321108279
[LightGBM] [Warning] feature_fraction is set=0.6655715429753413, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6655715429753413
[LightGBM] [Warning] bagging_fraction is set=0.921814321108279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.921814321108279


[I 2023-04-02 10:18:23,800] Trial 1 finished with value: 3.2819858129378385 and parameters: {'num_leaves': 223, 'feature_fraction': 0.6655715429753413, 'bagging_fraction': 0.921814321108279, 'min_child_samples': 64}. Best is trial 1 with value: 3.2819858129378385.


...

[I 2023-04-02 10:18:42,417] Trial 2 finished with value: 3.450672828041174 and parameters: {'num_leaves': 111, 'feature_fraction': 0.8145383041925325, 'bagging_fraction': 0.629002072782286, 'min_child_samples': 88}. Best is trial 1 with value: 3.2819858129378385.


[LightGBM] [Warning] feature_fraction is set=0.2603298347378491, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2603298347378491
[LightGBM] [Warning] bagging_fraction is set=0.9282843871644775, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9282843871644775
[LightGBM] [Warning] feature_fraction is set=0.2603298347378491, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2603298347378491
[LightGBM] [Warning] bagging_fraction is set=0.9282843871644775, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9282843871644775


[I 2023-04-02 10:18:52,522] Trial 3 finished with value: 3.4626153966363287 and parameters: {'num_leaves': 99, 'feature_fraction': 0.2603298347378491, 'bagging_fraction': 0.9282843871644775, 'min_child_samples': 19}. Best is trial 1 with value: 3.2819858129378385.


[LightGBM] [Warning] feature_fraction is set=0.22701771497267664, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.22701771497267664
[LightGBM] [Warning] bagging_fraction is set=0.6756084142201679, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6756084142201679
[LightGBM] [Warning] feature_fraction is set=0.22701771497267664, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.22701771497267664
[LightGBM] [Warning] bagging_fraction is set=0.6756084142201679, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6756084142201679


[I 2023-04-02 10:19:04,907] Trial 4 finished with value: 3.5985105632396004 and parameters: {'num_leaves': 178, 'feature_fraction': 0.22701771497267664, 'bagging_fraction': 0.6756084142201679, 'min_child_samples': 37}. Best is trial 1 with value: 3.2819858129378385.


In [6]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 223, 'feature_fraction': 0.6655715429753413, 'bagging_fraction': 0.921814321108279, 'min_child_samples': 64}


In [7]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)
        

[LightGBM] [Warning] feature_fraction is set=0.6655715429753413, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6655715429753413
[LightGBM] [Warning] bagging_fraction is set=0.921814321108279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.921814321108279


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001898E2510D0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.921814321108279,
                               feature_fraction=0.6655715429753413,
                               min_child_samples=64, num_leaves=223))])

In [8]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae = :.4f}')

test_mae = 2.5317


In [9]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 3979,
    predictions=pd.Series(predictions)
)

In [10]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 200,
    predictions=pd.Series(predictions)
)

In [11]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 1000,
    predictions=pd.Series(predictions)
)

In [12]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 900,
    predictions=pd.Series(predictions)
)

In [13]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 450,
    predictions=pd.Series(predictions)
)

In [14]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 6000,
    predictions=pd.Series(predictions)
)